In [3]:
import json
import pandas as pd

,id,text
0,1,면이 먹기 싫어 주문한 짬뽕밥은 면도 40%정도 있다. 밥은 .... 밥인지 떡인지...
1,2,어쩌다보니 연이틀 오게됐는데.. 원래 금액이면 좀 고민을 해 볼 것 같지만 차돌짬뽕...
2,3,오랜만에 갔는데 예전보다 맛은 좀 좋아진 것 같은데 밥 양이 좀 적어진듯.. 무난한...
3,4,크기를 비교할 게 같이 찍히지 않아 애매한데 곱배기고 양은 충분합니다. 유튜브 내꺼...
4,5,T- day 할인 쿠폰 받아서 간거라 큰 기대 안했는데 여기 짜장 예술이네요 😝 너...
5,6,어제가 특이했던 건지 확인차 이틀 연속 방문. 비오는 늦은 오후지만 사람이 꽤 있고...
6,7,여긴 주변대비 그냥 젤 맛있음
7,8,퇴근하고 기분 좋게 짜장면 3900원 이벤트 먹으러 왔는데 깐풍기 시켰는데 거의 안...
8,9,몇년 전 매장 냄새 민원 접수된 것부터 여전히 불쾌한 주방 사람들까지.
9,10,매번 먹어봐도 맛있게 잘 해주는 지점.


In [18]:
with open("naver_review.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data["reviews"])
for item in df:
    print(df[item])

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: id, dtype: int64
0    면이 먹기 싫어 주문한 짬뽕밥은 면도 40%정도 있다. 밥은 .... 밥인지 떡인지...
1    어쩌다보니 연이틀 오게됐는데.. 원래 금액이면 좀 고민을 해 볼 것 같지만 차돌짬뽕...
2    오랜만에 갔는데 예전보다 맛은 좀 좋아진 것 같은데 밥 양이 좀 적어진듯.. 무난한...
3    크기를 비교할 게 같이 찍히지 않아 애매한데 곱배기고 양은 충분합니다. 유튜브 내꺼...
4    T- day 할인 쿠폰 받아서 간거라 큰 기대 안했는데 여기 짜장 예술이네요 😝 너...
5    어제가 특이했던 건지 확인차 이틀 연속 방문. 비오는 늦은 오후지만 사람이 꽤 있고...
6                                     여긴 주변대비 그냥 젤 맛있음
7    퇴근하고 기분 좋게 짜장면 3900원 이벤트 먹으러 왔는데 깐풍기 시켰는데 거의 안...
8              몇년 전 매장 냄새 민원 접수된 것부터 여전히 불쾌한 주방 사람들까지.
9                                매번 먹어봐도 맛있게 잘 해주는 지점.
Name: text, dtype: object


In [2]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

/Users/apple/Softeer-DE/W2/w2m5_team/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [4]:
def sentiment_bert(text):
    result = sentiment_model(text)[0]
    stars = int(result["label"][0])
    return stars
    # return "positive" if stars >= 4 else "negative"

In [5]:
import re

def clean_text(text):
    text = re.sub(r"[^\w\s가-힣]", " ", text)   # 이모지, 특수문자 제거
    text = re.sub(r"\s+", " ", text).strip()   # 공백 정리
    return text

In [6]:
import kss

def split_sentences(text):
    return kss.split_sentences(text)


In [7]:
def preprocess_review(text):
    cleaned = clean_text(text)
    sentences = split_sentences(cleaned)
    return sentences

In [12]:
df["sentences"] = df["text"].apply(preprocess_review)

In [9]:
df.head()

,id,text,sentences
0,1,면이 먹기 싫어 주문한 짬뽕밥은 면도 40%정도 있다. 밥은 .... 밥인지 떡인지...,[면이 먹기 싫어 주문한 짬뽕밥은 면도 40 정도 있다 밥은 밥인지 떡인지 식감도 ...
1,2,어쩌다보니 연이틀 오게됐는데.. 원래 금액이면 좀 고민을 해 볼 것 같지만 차돌짬뽕...,[어쩌다보니 연이틀 오게됐는데 원래 금액이면 좀 고민을 해 볼 것 같지만 차돌짬뽕도...
2,3,오랜만에 갔는데 예전보다 맛은 좀 좋아진 것 같은데 밥 양이 좀 적어진듯.. 무난한...,[오랜만에 갔는데 예전보다 맛은 좀 좋아진 것 같은데 밥 양이 좀 적어진듯 무난한 ...
3,4,크기를 비교할 게 같이 찍히지 않아 애매한데 곱배기고 양은 충분합니다. 유튜브 내꺼...,[크기를 비교할 게 같이 찍히지 않아 애매한데 곱배기고 양은 충분합니다 유튜브 내꺼...
4,5,T- day 할인 쿠폰 받아서 간거라 큰 기대 안했는데 여기 짜장 예술이네요 😝 너...,"[T day 할인 쿠폰 받아서 간거라 큰 기대 안했는데 여기 짜장 예술이네요, 너무..."


In [15]:
df["star"] = df["sentences"].apply(sentiment_bert)
df["star_before"] = df["text"].apply(sentiment_bert)

In [16]:
df

,id,text,sentences,star,star_before
0,1,면이 먹기 싫어 주문한 짬뽕밥은 면도 40%정도 있다. 밥은 .... 밥인지 떡인지...,[면이 먹기 싫어 주문한 짬뽕밥은 면도 40 정도 있다 밥은 밥인지 떡인지 식감도 ...,2,2
1,2,어쩌다보니 연이틀 오게됐는데.. 원래 금액이면 좀 고민을 해 볼 것 같지만 차돌짬뽕...,[어쩌다보니 연이틀 오게됐는데 원래 금액이면 좀 고민을 해 볼 것 같지만 차돌짬뽕도...,2,3
2,3,오랜만에 갔는데 예전보다 맛은 좀 좋아진 것 같은데 밥 양이 좀 적어진듯.. 무난한...,[오랜만에 갔는데 예전보다 맛은 좀 좋아진 것 같은데 밥 양이 좀 적어진듯 무난한 ...,2,2
3,4,크기를 비교할 게 같이 찍히지 않아 애매한데 곱배기고 양은 충분합니다. 유튜브 내꺼...,[크기를 비교할 게 같이 찍히지 않아 애매한데 곱배기고 양은 충분합니다 유튜브 내꺼...,2,2
4,5,T- day 할인 쿠폰 받아서 간거라 큰 기대 안했는데 여기 짜장 예술이네요 😝 너...,"[T day 할인 쿠폰 받아서 간거라 큰 기대 안했는데 여기 짜장 예술이네요, 너무...",2,2
5,6,어제가 특이했던 건지 확인차 이틀 연속 방문. 비오는 늦은 오후지만 사람이 꽤 있고...,[어제가 특이했던 건지 확인차 이틀 연속 방문 비오는 늦은 오후지만 사람이 꽤 있고...,2,2
6,7,여긴 주변대비 그냥 젤 맛있음,[여긴 주변대비 그냥 젤 맛있음],3,3
7,8,퇴근하고 기분 좋게 짜장면 3900원 이벤트 먹으러 왔는데 깐풍기 시켰는데 거의 안...,[퇴근하고 기분 좋게 짜장면 3900원 이벤트 먹으러 왔는데 깐풍기 시켰는데 거의 ...,1,1
8,9,몇년 전 매장 냄새 민원 접수된 것부터 여전히 불쾌한 주방 사람들까지.,[몇년 전 매장 냄새 민원 접수된 것부터 여전히 불쾌한 주방 사람들까지],1,1
9,10,매번 먹어봐도 맛있게 잘 해주는 지점.,[매번 먹어봐도 맛있게 잘 해주는 지점],5,5
